# pre IRC

## outline:

- pre IRC
    - load mat (the neural recording file with behavior)
    - convert to IRC input (state, action, task)
- IRC (terminal script)


## prepare

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../privateconfig'))
resdir = Path(config['Datafolder']['data'])
workdir = Path(config['Codefolder']['workspace'])
os.chdir(workdir)

In [3]:
from scipy.io import loadmat
import numpy as np
from notification import notify
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.ndimage import gaussian_filter1d
import pickle
import pandas as pd
from plot_ult import *
import warnings
warnings.filterwarnings('ignore')

c:\Users\24455\miniconda3\envs\lab\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\24455\miniconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\24455\miniconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
# const
bin_size = 17 # how many bin of DT. about 0.1 s
num_bins = 24 # how many bins to use. use 2.4 s and discard the long trials.
monkey_height = 10
DT = 0.006 # DT for raw data
reward_boundary = 65
areas = ['PPC', 'PFC', 'MST']
t_total = 24
fontsize = 7; lw = 1
worldscale =200

m = 'm51'
folder='m51_mat_ruiyi'
dens=[0.0001, 0.0005, 0.001,  0.005 ]
idensity=3
density=dens[idensity]
locals().update({m: {}})
figure_path = resdir/'figures'
# datapaths = [i for i in Path(resdir/'mat_ruiyi').glob(f'{m}*.mat')]
datapaths=[i for i in Path(resdir/folder).glob(f'{m}*.mat')]
datapaths

[WindowsPath('C:/Users/24455/Desktop/repo/mkdata/m51_mat_ruiyi/m51s38.mat'),
 WindowsPath('C:/Users/24455/Desktop/repo/mkdata/m51_mat_ruiyi/m51s40.mat'),
 WindowsPath('C:/Users/24455/Desktop/repo/mkdata/m51_mat_ruiyi/m51s41.mat'),
 WindowsPath('C:/Users/24455/Desktop/repo/mkdata/m51_mat_ruiyi/m51s42.mat'),
 WindowsPath('C:/Users/24455/Desktop/repo/mkdata/m51_mat_ruiyi/m51s43.mat')]

In [5]:
# helper functions

def set_violin_plot(vp, facecolor, edgecolor, linewidth=1, alpha=1, ls='-', hatch=r''):
    plt.setp(vp['bodies'], facecolor=facecolor, edgecolor=edgecolor, 
             linewidth=linewidth, alpha=alpha ,ls=ls, hatch=hatch)
    plt.setp(vp['cmins'], facecolor=facecolor, edgecolor=edgecolor, 
             linewidth=linewidth, alpha=alpha)
    plt.setp(vp['cmaxes'], facecolor=facecolor, edgecolor=edgecolor, 
             linewidth=linewidth, alpha=alpha)
    plt.setp(vp['cbars'], facecolor=facecolor, edgecolor=edgecolor, 
             linewidth=linewidth, alpha=alpha)
    
    linecolor = 'k' if facecolor == 'None' else 'snow'
    if 'cmedians' in vp:
        plt.setp(vp['cmedians'], facecolor=linecolor, edgecolor=linecolor, 
                 linewidth=linewidth, alpha=alpha)
    if 'cmeans' in vp:
        plt.setp(vp['cmeans'], facecolor=linecolor, edgecolor=linecolor, 
                 linewidth=linewidth, alpha=alpha)
       
        
def downsample(data, bin_size=20):
    num_bin = data.shape[0] // bin_size
    data_ = data[:bin_size * num_bin]
    data_ = data_.reshape(num_bin, bin_size, data.shape[-1])
    data_ = np.nanmean(data_, axis=1)
    return data_




def convert_location_to_angle(gaze_r, gaze_x, gaze_y, body_theta, body_x, body_y, hor_theta_eye, ver_theta_eye,DT=DT, remove_pre=True):
    '''
        convert the world overhead view location of the 'gaze' location to eye coord. 

        gaze location, the target
        gaze_r, relative distance
        gaze_x, gaze location x
        gaze_y,

        body_theta, heading direction
        body_x, monkey location x
        body_y, 

        hor_theta_eye, actual eye location in eye coord. used here to remove pre saccade (when monkey hasnt seen the target yet)
        ver_theta_eye
    '''

    #hor_theta = -np.rad2deg(np.arctan2(-(gaze_x - body_x), gaze_y - body_y) - (body_theta-np.deg2rad(90))).reshape(-1, 1) 
    hor_theta = -np.rad2deg(np.arctan2(-(gaze_x - body_x), np.sqrt((gaze_y - body_y)**2 + monkey_height**2))
                            - (body_theta-np.deg2rad(90))).reshape(-1, 1) 
    overshoot_idx = np.where(((gaze_x - body_x) * gaze_x < 0) | (gaze_y < body_y)
                             #| (abs(hor_theta.flatten()) > 60)
                            )[0]
    
    if overshoot_idx.size > 0:
        hor_theta[overshoot_idx[0]:] = np.nan

    k = -1 / np.tan(body_theta); b = body_y - k * body_x
    gaze_r_sign = (k * gaze_x + b < gaze_y).astype(int)
    gaze_r_sign[gaze_r_sign == 0] = -1
    ver_theta = -np.rad2deg(np.arctan2(monkey_height, gaze_r_sign * gaze_r)).reshape(-1, 1)
    overshoot_idx = np.where((gaze_r_sign < 0)
                             #| (abs(ver_theta.flatten()) > 60)
                            )[0]
    if overshoot_idx.size > 0:
        ver_theta[overshoot_idx[0]:] = np.nan
        
    # detect saccade
    if remove_pre:
        if hor_theta_eye.size > 2:
            saccade = np.sqrt((np.gradient(hor_theta_eye) / DT)**2 + 
                            (np.gradient(ver_theta_eye) / DT)**2)
            saccade_start_idx = np.where(saccade > 100)[0]
            saccade_start_idx = saccade_start_idx[0] + 16 if saccade_start_idx.size > 0 else None

            hor_theta[:saccade_start_idx] = np.nan
            ver_theta[:saccade_start_idx] = np.nan
        
    return hor_theta, ver_theta


def compute_error(data1, data2, mask):
    #data1 = data1[~mask]; data2 = data2[~mask]
    #corr = np.corrcoef(data1, data2)
    error = abs(data1 - data2)
    
    rng = np.random.default_rng(seed=0)
    data1_ = data1.copy(); data2_ = data2.copy()
    rng.shuffle(data1_); rng.shuffle(data2_)
    error_shuffle = abs(data1_ - data2_)
    return error



## load raw data (same as ruiyi script)

In [6]:
# load raw data

for idx, datapath in enumerate(datapaths):
    if datapath.stem[-1].isalpha():
        continue
    data = loadmat(datapath)
    eval(m)[datapath.stem] = data
    notify(datapath)
    
notify('all done! loaded')

In [7]:
m_extracted_continuous = {}; m_downsampled = {}; m_errors = {}
for key, data in eval(m).items():
    if key[-1].isalpha():
        continue
        
    trials_behv = data['trials_behv'][0]
    trials_units = data['units'][0]
    units_area = np.array([v[0] for v in trials_units['brain_area']])
    
    Ydownsampled = []
    Y = []
    trials_error = []; trials_error_sign = []; trials_target_angle = []; trials_target_distance = []
    for trial_idx, trial_behv in enumerate(trials_behv):
        
        trial_ts = trial_behv['continuous']['ts'][0][0].reshape(-1)
        t_mask = (trial_ts > 0) & (~np.isnan(trial_behv['continuous']['ymp'][0][0].reshape(-1)))
        t_mask &= trial_ts < trial_behv['events']['t_stop'][0][0].reshape(-1)
        if t_mask.sum() > 0:
            t_mask[np.where(t_mask == True)[0][0]] = False # remove the first data point to avoid downsample error
        
        # get Y
        mx = trial_behv['continuous']['xmp'][0][0][t_mask]
        my = trial_behv['continuous']['ymp'][0][0][t_mask]
        fx = trial_behv['continuous']['xfp'][0][0][t_mask]
        fy = trial_behv['continuous']['yfp'][0][0][t_mask]
        sx = np.ones_like(fx); sy = np.ones_like(fy)
        if my.size > 0:
            fx = np.ones_like(fx) * fx[0]
            fy = np.ones_like(fy) * fy[0]
            sx *= mx[-1]; sy *= my[-1]
            my = my + 30; fy = fy + 30; sy = sy + 30
        
        dx = fx - mx; dy = fy - my
        rel_dist = np.sqrt(dx**2 + dy**2); rel_ang = np.rad2deg(np.arctan2(dy, dx))
        rel_dist_stop = np.sqrt((sx - mx)**2 + (sy - my)**2)
        
        if my.size > 0:
            trials_error.append(rel_dist[-1][0])
            trials_error_sign.append(rel_dist[-1][0])
            trials_target_angle.append(np.rad2deg(np.arctan2(fy, fx))[-1][0] - 90)
            trials_target_distance.append(np.sqrt(fx**2 + fy**2)[-1][0])
            
        else:
            trials_error.append(np.nan)
            trials_error_sign.append(np.nan)
            trials_target_angle.append(np.nan)
            trials_target_distance.append(np.nan)
        
        if my.size > 0:
            d1 = np.sqrt(fx**2 + fy**2)
            r1 = (fx**2 + fy**2) / (2*fx)
            radian1 = 2 * r1 * np.arcsin(d1 / (2 * r1))

            d2 = np.sqrt(mx**2 + my**2)
            r2 = (mx**2 + my**2) / (2*mx + 1e-8)
            radian2 = 2 * r2 * np.arcsin(d2 / (2 * r2 + 1e-8))

            sign = np.ones_like(rel_dist)
            sign[radian2 < radian1] = -1
            rel_dist = sign * rel_dist
            trials_error_sign[-1] = rel_dist[-1][0]
        
        abs_dist = np.sqrt(mx**2 + my**2); abs_ang = np.rad2deg(np.arctan2(my, mx))

        hor_theta = trial_behv['continuous']['yre'][0][0][t_mask]
        ver_theta = trial_behv['continuous']['zre'][0][0][t_mask]
        mw = -trial_behv['continuous']['w'][0][0][t_mask].reshape(-1)
        body_theta = np.deg2rad(np.cumsum(mw) * DT + 90)
        body_x, body_y = mx.reshape(-1), my.reshape(-1)
        
        hor_theta_, ver_theta_ = convert_location_to_angle(abs(rel_dist).reshape(-1), fx.reshape(-1), fy.reshape(-1),
                                                           body_theta, body_x, body_y, 
                                                           hor_theta.reshape(-1), ver_theta.reshape(-1))
        
               
        mv = trial_behv['continuous']['v'][0][0][t_mask].reshape(-1,1)
        mw = trial_behv['continuous']['w'][0][0][t_mask].reshape(-1,1)
        if t_mask.sum() * DT > 3.5 or t_mask.sum() * DT < 0.6 or mv.max() < 50 or \
            abs_dist[-1] < np.sqrt(fx**2 + fy**2)[-1] * 0.3:
            continue

        target_variable = np.hstack([rel_dist, rel_ang, abs_dist, abs_ang,
                                     hor_theta, ver_theta, hor_theta_, ver_theta_,
                                     fx, fy, mx, my, mv, mw])
        Y.append(target_variable)
        target_variable = downsample(target_variable, bin_size=bin_size)
        

        # [0.0001 0.0005 0.001  0.005 ]
        if trial_behv['prs'][0][0]['floordensity'] != density:
            continue
        
        Ydownsampled.append([trial_idx, target_variable])
   
    m_extracted_continuous[key + 'Y'] = Y
    m_downsampled[key + 'Ydownsampled'] = Ydownsampled

    m_errors[key + 'error'] = trials_error; m_errors[key + 'error_sign'] = trials_error_sign
    m_errors[key + 'target_angle'] = trials_target_angle; m_errors[key + 'target_distance'] = trials_target_distance
    
eval(m).update(m_downsampled); eval(m).update(m_extracted_continuous); eval(m).update(m_errors)
del m_downsampled, m_extracted_continuous, m_errors

## process the Y downsampled to irc input data (state, action, task)

In [8]:
irc_input_data={}

for key, data in eval(m).items():
    if not key.endswith('Ydownsampled'):
        continue
    states, actions, tasks=[],[],[]
    
    print(key)
    y=eval(m)[key]
    trial_idces, yy = zip(*y)

    for itrial in range(len(trial_idces)):
        rel_dist, rel_ang, abs_dist, abs_ang,hor_theta, ver_theta, hor_theta_, ver_theta_,fx, fy, mx, my, mv, mw=zip(*yy[itrial])
        # task
        taskx = (fx[0] - mx[0]).astype('float32'); tasky = (fy[0] - my[0]).astype('float32')
        tasks.append([tasky/worldscale,taskx/worldscale])
        # actions
        trialaction=np.stack([mv,mw]).T
        trialaction[:,0]=trialaction[:,0]/worldscale # v need reduce scale
        trialaction[:,1]=trialaction[:,1]/180*pi
        actions.append(trialaction.astype('float32'))

        # states from run the actions
        px, py, heading, v, w = 0,0,0,0,0
        log=[]
        for a in trialaction:
            px, py, heading, v, w=state_step2(px, py, heading, v, w, a, dt=0.1,userad=True)
            log.append([px, py, heading, v, w])
        px, py, heading, v, w=state_step2(px, py, heading, v, w, a, dt=0.1,userad=True)
        log.append([px, py, heading, v, w])
        trialstates=np.array(log)[1:]
        
        states.append(trialstates.astype('float32'))

    irc_input_data[key + '_irc']=(states, actions, tasks)

eval(m).update(irc_input_data)
del irc_input_data

m51s38Ydownsampled
m51s40Ydownsampled
m51s41Ydownsampled
m51s42Ydownsampled
m51s43Ydownsampled


In [9]:
all_states, all_actions, all_task=[],[],[]

for k,v in eval(m).items():
    if not k.endswith('irc'):
        continue
    print(k) # key

    sess_states, sess_actions, sess_task=v
    all_states+=sess_states
    all_actions+=sess_actions
    all_task+=sess_task

print(len(all_states))

m51s38Ydownsampled_irc
m51s40Ydownsampled_irc
m51s41Ydownsampled_irc
m51s42Ydownsampled_irc
m51s43Ydownsampled_irc
1192


In [10]:
# save the pre irc data

with open(resdir/folder/(f'preirc_den_{idensity}'), 'wb+') as f:
    pickle.dump((all_states, all_actions, all_task), f)
print(resdir/folder/(f'preirc_den_{idensity}'))


C:\Users\24455\Desktop\repo\mkdata\m51_mat_ruiyi\preirc_den_3


## 